### Crawler

In [163]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient['1001']
mydb['track_docs'].delete_many({})
mydb['artist_docs'].delete_many({})
mydb['played_docs'].delete_many({})
mydb['tracklist_docs'].delete_many({})
mydb['sequential_docs'].delete_many({})
print(len(list(mydb['track_docs'].find({}))))
print(len(list(mydb['artist_docs'].find({}))))
print(len(list(mydb['played_docs'].find({}))))

0
0
0


In [ ]:
def tracklist_track_data(html):
    
    '''
    Extract track related data from set
    '''
    track_docs = {}
    index = 0
    while find_str(html, 'tracknumber_value">', index) != -1:

        index = find_str(html, 'tracknumber_value">', index)
        #print(index)
        track_chunk = html[index:].split('<br>')[0]

        # Extract track number
        track_num = track_chunk[:22].split('<')[0].strip('tracknumber_value">')
        #print('Track Number:', track_num)

        # Extract track information
        chunk_index = 0
        chunk_index = find_str(track_chunk, 'meta itemprop="name" content=', chunk_index)
        extracted_value = track_chunk[chunk_index:].strip('meta itemprop="name" content=').split('>')[0].strip('"')
        clean_string = fix_decoding_errors(extracted_value)
        #print(clean_string)
        
        if len(clean_string) > 1:
            try:
                artist_list, track, remixer = parse_track_and_artist(clean_string)
            except:
                artist_list, track, remixer = None, None, None
        else:
            artist_list, track, remixer = None, None, None
            
        # Avoid ID's for now
        if artist_list is None:
            pass
        # If track info pull failed then pass
        elif (('ID' in artist_list) or ('ID' in track)):
            pass
        else:
            
            # Tends to be multiple artists so artists parsed to list even if only one
            for artist in artist_list:
                
                #print('Artist:',artist)
                #print('Track:', track)
                #print('Remixer:', remixer)

                # Extract artist page
                artist_index = 0
                artist_index = find_str(track_chunk, 'title="open artist page"', artist_index)
                if artist_index != -1:
                    try:
                        artist_url = track_chunk[artist_index:].split('class')[1].split('href="')[1].rstrip('" ')
                        artist_url = 'https://www.1001tracklists.com' + artist_url
                        #print('Aritst url:', artist_url)
                    except:
                        artist_url = 'N/A'
                else:
                    artist_url = 'N/A'

                # Extract remixer page (if exists)
                if remixer != 'N/A':
                    remixer_index = find_str(track_chunk, 'title="open remixer artist page"', artist_index)
                    if remixer_index != -1:
                        try:
                            remixer_url = track_chunk[remixer_index:].split('class')[1].split('href="')[1].rstrip('" ')
                            remixer_url = 'https://www.1001tracklists.com' + remixer_url
                            #print('Remixer url:', remixer_url)
                        except:
                            remixer_url = 'N/A'
                    else:
                        remixer_url = 'N/A'
                else:
                    remixer_url = 'N/A'

                # Extract track page
                track_index = 0
                track_index = find_str(track_chunk, 'title="open track page"', artist_index)
                if track_index != -1:
                    try:
                        track_url = track_chunk[track_index:].split('class')[1].split('href="')[1].split('"')[0]
                        track_url = 'https://www.1001tracklists.com' + track_url
                        #print('track url:', track_url)
                    except:
                        track_url = 'N/A'
                else:
                    track_url = 'N/A'

                track_doc = {\
                            'track_num': track_num,
                            'artist': artist.strip(' '),
                            'artist_url': artist_url.strip(' '),
                            'name': track.strip(' '),
                            'track_url': track_url.strip(' '),
                            'remixer': remixer.strip(' '),
                            'remixer_url': remixer_url.strip(' ')
                            }
                track_docs[track_num] = track_doc
                #print('\n\n\n')

    return track_docs

def build_artist_edges(url_doc, url):
    '''
    Build artist set-adjacency docs -- order n^2.
    Dont iterate over full set twice since will be considered non-directional
    '''
    all_tracks = {}
    count = 0
    these_tracks = list(url_doc['track_docs'].values())
    for i in range(len(these_tracks)):
        for j in range(i,len(these_tracks)):

            track = these_tracks[i]
            other_track = these_tracks[j]

            first_artist = track['artist']
            second_artist = other_track['artist']

            if first_artist != second_artist:
                _id = '_'.join([url,first_artist,second_artist])
                all_tracks[_id] = \
                                    {
                                    #'_id': _id,
                                    'artist1': first_artist,
                                    'artist2': second_artist,
                                    'url': url
                                    }
    return all_tracks

def build_track_edges(track_docs, url):
    '''
    Build track set-adjacency docs -- order n^2.
    Dont iterate over full set twice since will be considered non-directional
    '''
    edge_docs = {}
    keys = sorted(list(track_docs.keys()))
    for i in range(len(keys)):
        for j in range(i, len(keys)):
            
            key = keys[i]
            other_key = keys[j]
            
            if key != other_key:
                _id = '_'.join([url,'_'.join(key),'_'.join(other_key)])
                edge_docs[_id] = \
                                    {
                                    #'_id': _id,
                                    'track1': key,
                                    'track2': other_key,
                                    'url': url
                                    }
    return edge_docs
                
def build_sequential_track_edges(track_docs, url):
    '''
    Allows for later "next track lookup" functionality
    
    '''
    enumerated_tracks = [(track_docs[key]['track_num'], key) for key in list(track_docs.keys())]
    enumerated_tracks = sorted(enumerated_tracks, key=lambda x: x[0])
    
    seq_docs = {}
    for track_idx in range(len(enumerated_tracks)-1):
        _id = '_'.join(\
                      [\
                       url,\
                       '_'.join(enumerated_tracks[track_idx][1]),\
                       '_'.join(enumerated_tracks[track_idx+1][1])
                      ]
                    )
        seq_docs[_id] = \
                       {
                       #'_id': _id,
                       'url': url,
                       'first_track': enumerated_tracks[track_idx][1],
                       'second_track': enumerated_tracks[track_idx+1][1],
                       'first_position': str(enumerated_tracks[track_idx][0]),
                       'second_position': str(enumerated_tracks[track_idx+1][0]),   
                       }
    return seq_docs

def build_played_playedby_edge(url_doc, url):
    '''
    Allows you to map who plays who.
    I think it would be interesting to study directional graphs from this.
    
    '''
    dj_name = url_doc['dj_name']
    dj_url = url_doc['dj_url']
    
    if (dj_name == 'N/A') or (dj_url == 'N/A'):
        return {}
    
    played_docs = {}
    for track_doc in list(url_doc['track_docs'].values()):
        _id = '_'.join([url,dj_name,track_doc['name']])
        played_docs[_id] = \
                          {
                          #'_id': _id,
                          'url': url,
                          'played_by': dj_name,
                          'played_by_url': dj_url,
                          'played': track_doc['name'],
                          'played_track_url': track_doc['track_url'],
                          'played_artist': track_doc['artist'],
                          'played_artist_url': track_doc['artist_url'],
                          'played_remixer': track_doc['remixer'],
                          'played_remixer_url': track_doc['remixer_url']
                          }
    return played_docs



class Crawler:
    
    def __init__(self, max_depth=1):
        
        import time
        import pymongo
        from urllib.request import Request, urlopen
        
        myclient = pymongo.MongoClient("mongodb://localhost:27017/")
        db = myclient['1001']
        
        self.url_html_map = db['url_html_map']
        self.tracklist_collection = db['tracklist_docs']
        self.played_collection = db['played_docs']
        self.track_collection = db['track_docs']
        self.artist_collection = db['artist_docs']
        self.sequential_collection = db['sequential_docs']
        
        self.stop_search = False
        
        self.urls = []
        self.max_depth = max_depth
        self.page_hash = {}
        self.tracklist_hash = {}
        
        self.track_docs = []
        self.played_docs = []
        self.artist_docs = []
        self.tracklist_docs = []
        self.sequential_docs = []

    def find_str(self, s, char, start_index=0):

        index = 0
        s = s[start_index+1:]
        if char in s:
            c = char[0]
            for ch in s:
                if ch == c:
                    if s[index:index+len(char)] == char:
                        return start_index + 1 + index
                index += 1
        return -1

    def request(self,url):

        req = Request(url,\
                      headers={'User-Agent': 'Mozilla/5.0'})
        html = str(urlopen(req).read())
        return html
    
    def parse(self, url, html): 

        url_doc = {}
        url_doc['url'] = url
        url_doc['html'] = html
        self.url_html_map.insert_one(url_doc)
        
        '''
        
        GET RID OF THIS LATER
        
        '''
        try:
            
            url_doc.update(tracklist_meta_data(html))
            url_doc.update(tracklist_general_information(html))

            track_docs = tracklist_track_data(html)
            url_doc['track_docs'] = track_docs

            track_edges = build_track_edges(track_docs, url).values()
            sequential_edges = build_sequential_track_edges(track_docs, url).values()
            played_edges = build_played_playedby_edge(url_doc, url).values()
            artist_edges = build_artist_edges(url_doc, url).values()

            self.tracklist_collection.insert_one(url_doc)
            for doc in track_edges:
                self.track_collection.insert_one(doc)
            for doc in artist_edges:
                self.artist_collection.insert_one(doc)
            for doc in played_edges:
                self.played_collection.insert_one(doc)
            for doc in sequential_edges:
                self.sequential_collection.insert_one(doc)

            self.played_docs.extend(played_edges)
            self.track_docs.extend(track_edges)
            self.tracklist_docs.append(url_doc)
            self.artist_docs.extend(artist_edges)        
            self.sequential_docs.extend(sequential_edges)

            print('Len played docs:', len(self.played_docs))
            print('Len sequential docs:', len(self.sequential_docs))
            print('Len track docs:', len(self.track_docs))
            print('Len tracklist docs:', len(self.tracklist_docs))
            print('Len artist docs:', len(self.artist_docs))

            with open('played_docs_crawl.pkl', 'wb') as f:
                pickle.dump(self.played_docs, f)
            with open('sequential_docs_crawl.pkl', 'wb') as f:
                pickle.dump(self.sequential_docs, f)
            with open('track_docs_crawl.pkl', 'wb') as f:
                pickle.dump(self.track_docs, f)
            with open('tracklist_docs_crawl.pkl', 'wb') as f:
                pickle.dump(self.tracklist_docs, f)
            with open('artist_docs_crawl.pkl', 'wb') as f:
                pickle.dump(self.artist_docs, f)
            with open('url_html_map_crawl.pkl', 'wb') as f:
                pickle.dump(self.page_hash, f)
                
        except:
            pass
    
    def crawl(self, url, depth):
        
        if len(self.tracklist_docs) == 5000:
            print('STOPPING SEARCH')
            self.stop_search = True
        
        print('Depth:', depth)
        if (depth == self.max_depth) or (self.stop_search):
            return
        
        # Check if already reached by search
        if self.page_hash.get(url, 0) == False:
            
            # Only sleep if about to request
            time.sleep(10)
            
            # Make http request
            try:
                html = self.request(url)
            except:
                return
            
            print(url)
            
            # Cache url-html map
            self.page_hash[url] = html
            
            # If html, parse and extract necessary data 
            if ('/tracklist/' in url):
                self.parse(url, html)
                    
            index = 0
            # Iterate over links found in html
            while self.find_str(html, 'a href="', index) != -1:
                
                # Extract url
                index = self.find_str(html, 'a href="', index)
                url_chunk = html[index:].split('"')[1]
                
                # Make sure it is either a referenced tracklist or 1001 page
                if ('/tracklist/' in url_chunk) and\
                   ('http' not in url_chunk) and\
                   ('#tlp' not in url_chunk):
                
                    self.urls.append(url)
                    new_page = 'https://www.1001tracklists.com' + url_chunk
                    self.crawl(new_page, depth + 1)
                
                if ('/dj/' in url_chunk) and\
                   ('http' not in url_chunk) and\
                   ('#tlp' not in url_chunk):
                
                    self.urls.append(url)
                    new_page = 'https://www.1001tracklists.com' + url_chunk
                    self.crawl(new_page, depth + 1)
                
                if ('www.1001tracklists.com' in url_chunk) and\
                   ('#tlp' not in url_chunk) and\
                   ('.xml' not in url_chunk):
                    
                    self.urls.append(url)
                    self.crawl(url_chunk, depth + 1)
                    
        return
        
    def start_crawl(self, startUrl):
        
        depth = 0
        self.crawl(startUrl, depth)
        
crawler = Crawler(max_depth=50)
crawler.start_crawl('https://www.1001tracklists.com/source/v7m7k3/the-anjunadeep-edition/index.html')

https://www.1001tracklists.com/tracklist/yx1xjj9/makj-revolution-radio-show-289-2018-11-19.html
Len played docs: 99240
Len sequential docs: 101116
Len track docs: 1923578
Len tracklist docs: 4030
Len artist docs: 1384909
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
Depth: 49
Depth: 48
Depth: 48
Depth: 47
Depth: 47
Depth: 46
Depth: 46
Depth: 46
Depth: 46
https://www.1001tracklists.com/tracklist/27wbw50k/makj-revolution-radio-show-288-2018-11-12.html
Len played docs: 99259
Len sequential docs: 101134
Len track docs: 1923749
Len tracklist docs: 4031
Len artist docs: 1385080
Depth: 47
Depth: 47
Depth: 47
https://www.1001tracklists.com/tracklist/28u3ubht/makj-revolution-radio-show-287-2018-11-05.html
Len played docs: 99278
Len sequential docs: 101152
Len track docs: 1923920
Len tracklist docs: 4032
Len artist docs: 1385251
Depth: 48
Depth: 48
Depth: 48
https://www.1001tracklists.com/tracklist/1r3v8v19/makj-revolution-radio-show-286-2018-10-29.html
Len played docs: 99294
Len s

Depth: 48
Depth: 48
https://www.1001tracklists.com/tracklist/f6zhgyt/frank-nitty-kryders-kryteria-radio-168-2019-01-09.html
Len played docs: 99849
Len sequential docs: 101713
Len track docs: 1931029
Len tracklist docs: 4059
Len artist docs: 1391766
Depth: 49
https://www.1001tracklists.com/dj/franknitty/index.html
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/jyk8cnk/swedish-egil-frank-nitty-groove-radio-international-1373-2019-03-20.html
Len played docs: 99866
Len sequential docs: 101729
Len track docs: 1931165
Len tracklist docs: 4060
Len artist docs: 1391902
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/1nbv957t/richard-vission-frank-nitty-swedish-egil-powertools-mixshow-2019-03-02.html
Len played d

Depth: 49
Depth: 49
https://www.1001tracklists.com/tracklist/6rr4kd9/danny-howard-mark-knight-bbc-radio-1-dance-anthems-2013-11-16.html
Len played docs: 100291
Len sequential docs: 102154
Len track docs: 1937845
Len tracklist docs: 4078
Len artist docs: 1397617
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
Depth: 49
Depth: 49
https://www.1001tracklists.com/tracklist/5mzxk49/danny-howard-fatboy-slim-bbc-radio-1-dance-anthems-2013-12-07.html
Len played docs: 100343
Len sequential docs: 102207
Len track docs: 1939276
Len tracklist docs: 4079
Len artist docs: 1398843
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
Depth: 49
Depth: 49
https://www.1001tracklists.com/tracklist/5btrj3k/danny-howard-dimitri-vegas-and-like-mike-bbc-radio-1-dance-anthems-2013-06-29.html
Len played docs: 100394
Len sequential docs: 102261
Len track docs: 1940761
Len trac

Depth: 49
Depth: 49
Depth: 49
Depth: 49
Depth: 48
https://www.1001tracklists.com/tracklist/2nx9qyg9/reuben-keeney-crizify-outhouse-022-2018-08-11.html
Len played docs: 100851
Len sequential docs: 102702
Len track docs: 1946633
Len tracklist docs: 4101
Len artist docs: 1404996
Depth: 49
Depth: 49
Depth: 49
Depth: 49
Depth: 48
https://www.1001tracklists.com/tracklist/x14h3s9/reuben-keeney-outhouse-021-2018-08-04.html
Len played docs: 100863
Len sequential docs: 102713
Len track docs: 1946699
Len tracklist docs: 4102
Len artist docs: 1405062
Depth: 49
Depth: 49
Depth: 49
Depth: 48
https://www.1001tracklists.com/tracklist/10s9y10k/reuben-keeney-latmun-outhouse-020-2018-07-28.html
Len played docs: 100875
Len sequential docs: 102724
Len track docs: 1946765
Len tracklist docs: 4103
Len artist docs: 1405128
Depth: 49
Depth: 49
https://www.1001tracklists.com/dj/latmun/index.html
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 

Len played docs: 101256
Len sequential docs: 103091
Len track docs: 1950353
Len tracklist docs: 4128
Len artist docs: 1408115
Depth: 47
Depth: 47
https://www.1001tracklists.com/dj/manpower/index.html
Depth: 48
Depth: 48
Depth: 48
https://www.1001tracklists.com/tracklist/xyu0rm9/man-power-juan-ramos-bis-radio-show-945-2018-07-03.html
Len played docs: 101283
Len sequential docs: 103118
Len track docs: 1950731
Len tracklist docs: 4129
Len artist docs: 1408445
Depth: 49
Depth: 49
Depth: 49
Depth: 48
https://www.1001tracklists.com/tracklist/1ft9k8pk/man-power-trax-magazine-284-2018-05-09.html
Len played docs: 101294
Len sequential docs: 103128
Len track docs: 1950786
Len tracklist docs: 4130
Len artist docs: 1408500
Depth: 49
Depth: 49
Depth: 48
https://www.1001tracklists.com/tracklist/2ffswuyt/b.traits-peggy-gou-julia-govor-bbc-radio-1-b.traits-show-2017-07-01.html
Len played docs: 101331
Len sequential docs: 103164
Len track docs: 1951452
Len tracklist docs: 4131
Len artist docs: 1409027


...

Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/1grqv9jt/piem-too-many-rules-podcast-006-2018-11-22.html
Len played docs: 112566
Len sequential docs: 114324
Len track docs: 2135050
Len tracklist docs: 4663
Len artist docs: 1540605
Depth: 50
Depth: 50
Depth: 50
Depth: 49
Depth: 49
https://www.1001tracklists.com/tracklist/zv0u0tt/piem-javi-bora-cessle-innit-trustnobody-show-021-2018-08-19.html
Len played docs: 112584
Len sequential docs: 114346
Len track docs: 2135303
Len tracklist docs: 4664
Len artist docs: 1540742
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/2m1p82vt/piem-the-next-evolution-027-2018-08-04.html
Len played docs: 112596
Len sequential docs: 114357
Len track docs: 2135369
Len tracklist docs: 4665
Len artist docs: 1540808
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/1xnqdgv9/piem-juany-bravo-trustnobody-show-020-2018-07-

Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/xyqgq61/mde-my-digital-enemy-byor-mde-radio-290-2018-05-25.html
Len played docs: 112978
Len sequential docs: 114718
Len track docs: 2138305
Len tracklist docs: 4690
Len artist docs: 1543383
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/bdufcc1/mde-my-digital-enemy-mde-radio-289-2018-05-15.html
Len played docs: 112990
Len sequential docs: 114729
Len track docs: 2138371
Len tracklist docs: 4691
Len artist docs: 1543449
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/w325vz1/mde-my-digital-enemy-mde-radio-288-2018-05-14.html
Len played docs: 113005
Len sequential docs: 114743
Len track docs: 2138476
Len tracklist docs: 4692
Len artist docs: 1543554
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 

Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/1p8lkf49/adam-foster-adam-foster-radio-026-2019-02-08.html
Len played docs: 113400
Len sequential docs: 115112
Len track docs: 2141425
Len tracklist docs: 4718
Len artist docs: 1546241
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/j0gjj2k/adam-foster-adam-foster-radio-025-2019-01-27.html
Len played docs: 113418
Len sequential docs: 115129
Len track docs: 2141578
Len tracklist docs: 4719
Len artist docs: 1546385
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/2j8tfbz9/adam-foster-adam-foster-radio-024-2019-01-19.html
Len played docs: 113434
Len sequential docs: 115144
Len track docs: 2141698
Len tracklist docs: 4720
Len artist docs: 1546492
Depth: 50
Depth: 50
Depth: 50
Depth: 49
https://www.1001tracklists.com/tracklist/v5zl9pt/adam-foster-adam-foster-radio-best-of-2018-2018-12-30.html
Len played docs: 113476
Len sequential docs: 115185
Len

https://www.1001tracklists.com/tracklist/2hbhfubk/black-caviar-exquisite-house-selections-vol.-2-2018-03-06.html
Len played docs: 113974
Len sequential docs: 115657
Len track docs: 2148292
Len tracklist docs: 4747
Len artist docs: 1552636
Depth: 49
Depth: 49
Depth: 48
https://www.1001tracklists.com/tracklist/1vsn8qtt/black-caviar-exquisite-house-selections-vol.-1-2017-12-13.html
Len played docs: 114000
Len sequential docs: 115682
Len track docs: 2148617
Len tracklist docs: 4748
Len artist docs: 1552923
Depth: 49
Depth: 49
Depth: 47
Depth: 47
https://www.1001tracklists.com/tracklist/1nbwtg59/richard-vission-lavelle-dupree-gettoblaster-diplo-powertools-mixshow-mixshow-2019-03-15.html
Len played docs: 114036
Len sequential docs: 115717
Len track docs: 2149247
Len tracklist docs: 4749
Len artist docs: 1553456
Depth: 48
Depth: 48
https://www.1001tracklists.com/dj/lavelledupree/index.html
Depth: 49
Depth: 49
https://www.1001tracklists.com/tracklist/xzq6b61/freshcobar-lavelle-dupree-discotheq

Depth: 49
https://www.1001tracklists.com/dj/spektre/index.html
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
Depth: 49
https://www.1001tracklists.com/tracklist/1q4ygdg9/moullinex-traxsource-live-140-2017-10-05.html
Len played docs: 114390
Len sequential docs: 116055
Len track docs: 2153009
Len tracklist docs: 4769
Len artist docs: 1556711
Depth: 50
Depth: 50
Depth: 50
Depth: 50
Depth: 49
Depth: 49
Depth: 48
Depth: 46
https://www.1001tracklists.com/tracklist/2dk18xm9/doorly-traxsource-live-183-2018-08-02.html
Len played docs: 114403
Len sequential docs: 116067
Len track docs: 2153087
Len tracklist docs: 4770
Len artist docs: 1556752


### Parser

- Should probably remove features from artist values
- and break up artists into multiple entries if there is like & or vs. or something

In [156]:
import ftfy
import urllib
from urllib.request import Request, urlopen

def find_str(s, char, start_index=0):
    '''
    Find substring char in string s. Found on internet, probably not efficient.
    
    '''
    index = 0
    s = s[start_index+1:]
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return start_index + 1 + index
            index += 1
    return -1

def extract_value(html, key_value):
    
    content_string = html.strip(key_value).split('>')[0].strip('"')
    return content_string

def fix_decoding_errors(string):
    '''
    Fix UTF-8 decoding issues. Probably need to find more systematic/thorough approach to this.
    
    REPLACE THIS WITH ftfy.fix_text() -- python package which should be one stop shop for fixes
    '''
    string = string.replace('&amp;','&')
    string = string.replace('&#39;',"'")
    string = string.replace('\\xc3\\xb6','o')
    string = string.replace('\\xc3\\xab','e')
    string = string.replace('\\xc3\\x9','u')
    string = string.replace('\\xc3\\xb8','o')
    string = string.replace("\\'","'")
    
    return ftfy.fix_text(string)

def parse_track_and_artist(track_string):
    '''
    Extract the artist, track name, and remixer (if any) from the standard formatting used by 1001.
    
    '''
    # Check if Remix/Bootleg/Edit and parse accordingly
    if ('Remix' in track_string) or ('Bootleg' in track_string) or ('Edit' in track_string):
        
        artist, track_remixer = [string.strip(' ') for string in track_string.split(' - ')]
        track_remixer = [string.strip(' ') for string in track_remixer.split('(')]
        
        if len(track_remixer) > 2:
            track = track_remixer[0]
            remixer = '('.join(track_remixer[1:])
        else:
            track, remixer = track_remixer
            remixer = remixer.rstrip('Remix)').strip(' ')
        
    # If not remix, then should follow standard layout "Artist Name - Track Name"
    # This layout is expressed explicitly in html
    else:
        
        artist, track = [string.strip(' ') for string in track_string.split(' - ')]
        remixer = 'N/A'

    # Check for multiple artists -- Big Room sets tend to have hella mashups
    # Sometimes there is more structured formatting to exploit i.e. (Artist1 vs. Artist2 - Track1 vs. Track2)
    # Not worrying about that now b/c big room sux
    if 'vs.' in artist:
        artist = artist.replace('vs.','&')
    if '&' in artist:
        artist = [a.strip(' ') for a in artist.split('&')]
    
    # Remove features
    # We could make features a separate field but for now just removing
    if isinstance(artist, str):
        if ('feat.' in artist) or ('ft.' in artist):
            artist = artist.split('feat.')[0].strip(' ')
            artist = artist.split('ft.')[0].strip(' ')
    if isinstance(artist, list):
            artist = [a.split('feat.')[0].split('ft.')[0].strip(' ') for a in artist]
        
    if isinstance(artist, list):
        return (artist, track, remixer)
    else:
        return ([artist], track, remixer)


def tracklist_meta_data(html):
    '''
    Extract meta data about tracklist/set.
    
    '''
    meta_data = {}
    
    # Extract set description
    index = 0
    start_term = 'meta name="description" content="'
    index = find_str(html, start_term, index)
    description = html[index:].split('>')[0]
    description = description.lstrip(start_term).rstrip('"')
    meta_data['description'] = description
    
    # Set creation date - This should probably be the point in time we use for building prediction data
    index = 0
    start_term = 'meta name="dcterms.created" content="'
    index = find_str(html, start_term, index)
    created = html[index:].split('>')[0]
    created = created.lstrip(start_term).rstrip('"')
    meta_data['created'] = created
    
    # Set last modified data
    index = 0
    start_term = 'meta name="dcterms.modified" content="'
    index = find_str(html, start_term, index)
    modified = html[index:].split('>')[0]
    modified = modified.lstrip(start_term).rstrip('"')
    meta_data['modified'] = modified
    
    return meta_data
    
def tracklist_general_information(html):
    '''
    Extract general info about tracklist/set.
    
    '''
    info_doc = {}
    index = 0
    start_term = 'General Information'
    index = find_str(html, start_term, index)
    info_chunk = html[index:].split('Most Liked Tracklists')[0]
    
    # Genres -- can use these to build genre-specific graphs
    style_index = 0
    style_index = find_str(info_chunk, 'Tracklist Musicstyle', style_index)
    styles = info_chunk[style_index:].split('id="tl_music_styles">')[1].split('</td>')[0]
    styles = [style.strip(' ') for style in styles.split(',')]
    info_doc['styles'] = styles
    
    # If 1001 recognizes the dj who played the set they link their dj page
    # Its my understanding dj pages are independent of artist pages -- we'll need to map these
    index = 0
    start_term = 'a href="/dj'
    index = find_str(html, start_term, index)
    if index != -1:
        dj_url = html[index:].split('class')[0].split('"')[1]
        dj_url = 'https://www.1001tracklists.com' + dj_url
        info_doc['dj_url'] = dj_url

        dj_name = html[index:].split('</a>')[0].split('>')[1]
        info_doc['dj_name'] = dj_name
    else:
        info_doc['dj_url'] = 'N/A'
        info_doc['dj_name'] = 'N/A'
        
    return info_doc
    
def tracklist_track_data(html):
    '''
    Extract track related data from set
    '''
    track_docs = {}
    index = 0
    while find_str(html, 'tracknumber_value">', index) != -1:

        index = find_str(html, 'tracknumber_value">', index)
        #print(index)
        track_chunk = html[index:].split('<br>')[0]
        #print(track_chunk)
        
        # Extract track number
        track_num = track_chunk[:22].split('<')[0].strip('tracknumber_value">')
        #print('Track Number:', track_num)

        # Extract track information
        chunk_index = 0
        chunk_index = find_str(track_chunk, 'meta itemprop="name" content=', chunk_index)
        extracted_value = track_chunk[chunk_index:].strip('meta itemprop="name" content=').split('>')[0].strip('"')
        clean_string = fix_decoding_errors(extracted_value)
        #print(clean_string)
        
        if len(clean_string) > 1:
            try:
                artist_list, track, remixer = parse_track_and_artist(clean_string)
            except:
                artist_list, track, remixer = None, None, None
        else:
            artist_list, track, remixer = None, None, None
            
        # Avoid ID's for now
        if artist_list is None:
            pass
        # If track info pull failed then pass
        elif (('ID' in artist_list) or ('ID' in track)): 
            pass
        else:
            
            # Tends to be multiple artists so artists parsed to list even if only one
            for artist in artist_list:
                
                #print('Artist:',artist)
                #print('Track:', track)
                #print('Remixer:', remixer)

                # Extract artist page
                artist_index = 0
                artist_index = find_str(track_chunk, 'title="open artist page"', artist_index)
                if artist_index != -1:
                    artist_url = track_chunk[artist_index:].split('class')[1].split('href="')[1].rstrip('" ')
                    #print('Aritst url:', artist_url)
                    artist_url = 'https://www.1001tracklists.com' + artist_url
                else:
                    artist_url = 'N/A'

                # Extract remixer page (if exists)
                if remixer != 'N/A':
                    remixer_index = find_str(track_chunk, 'title="open remixer artist page"', artist_index)
                    if remixer_index != -1:
                        remixer_url = track_chunk[remixer_index:].split('class')[1].split('href="')[1].rstrip('" ')
                        #print('Remixer url:', remixer_url)
                        remixer_url = 'https://www.1001tracklists.com' + remixer_url
                    else:
                        remixer_url = 'N/A'
                else:
                    remixer_url = 'N/A'

                # Extract track page
                track_index = 0
                track_index = find_str(track_chunk, 'title="open track page"', artist_index)
                if track_index != -1:
                    track_url = track_chunk[track_index:].split('class')[1].split('href="')[1].split('"')[0]
                    #print('track url:', track_url)
                    track_url = 'https://www.1001tracklists.com' + track_url
                else:
                    track_url = 'N/A'

                track_doc = {\
                            'track_num': track_num,
                            'artist': artist.strip(' '),
                            'artist_url': artist_url.strip(' '),
                            'name': track.strip(' '),
                            'track_url': track_url.strip(' '),
                            'remixer': remixer.strip(' '),
                            'remixer_url': remixer_url.strip(' ')
                            }
                track_docs[track_num] = track_doc
                #print('\n\n\n')

    return track_docs

def build_artist_edges(url_doc, url):
    '''
    Build artist set-adjacency docs -- order n^2.
    Dont iterate over full set twice since will be considered non-directional
    '''
    all_tracks = []
    these_tracks = list(url_doc['track_docs'].values())
    for i in range(len(these_tracks)):
        for j in range(i,len(these_tracks)):

            track = these_tracks[i]
            other_track = these_tracks[j]

            first_artist = track['artist']
            second_artist = other_track['artist']

            if first_artist != second_artist:
                all_tracks.append(\
                                {
                                'artist1': first_artist,
                                'artist2': second_artist,
                                'url': url
                                }
                            )
    return all_tracks

def build_track_edges(track_docs, url):
    '''
    Build track set-adjacency docs -- order n^2.
    Dont iterate over full set twice since will be considered non-directional
    '''
    edge_docs = []
    keys = sorted(list(track_docs.keys()))
    for i in range(len(keys)):
        for j in range(i, len(keys)):
            
            key = keys[i]
            other_key = keys[j]
            
            if key != other_key:
                edge_docs.append(\
                                {
                                'track1': key,
                                'track2': other_key,
                                'url': url
                                }
                            )
    return edge_docs
                
def build_sequential_track_edges(track_docs, url):
    '''
    Allows for later "next track lookup" functionality
    
    '''
    enumerated_tracks = [(track_docs[key]['track_num'], key) for key in list(track_docs.keys())]
    enumerated_tracks = sorted(enumerated_tracks, key=lambda x: x[0])
    
    seq_docs = []
    for track_idx in range(len(enumerated_tracks)-1):
        seq_docs.append(\
                       {
                       'url': url,
                       'first_track': enumerated_tracks[track_idx][1],
                       'second_track': enumerated_tracks[track_idx+1][1],
                       'first_position': enumerated_tracks[track_idx][0],
                       'second_position': enumerated_tracks[track_idx+1][0],
                       }
                    )
    return seq_docs

def build_played_playedby_edge(url_doc, url):
    '''
    Allows you to map who plays who.
    I think it would be interesting to study directional graphs from this.
    
    '''
    dj_name = url_doc['dj_name']
    dj_url = url_doc['dj_url']
    
    if (dj_name == 'N/A') or (dj_url == 'N/A'):
        return []
    
    played_docs = []
    for track_doc in list(url_doc['track_docs'].values()):
        
        played_docs.append(\
                          {
                          'url': url,
                          'played_by': dj_name,
                          'played_by_url': dj_url,
                          'played': track_doc['name'],
                          'played_track_url': track_doc['track_url'],
                          'played_artist': track_doc['artist'],
                          'played_artist_url': track_doc['artist_url'],
                          'played_remixer': track_doc['remixer'],
                          'played_remixer_url': track_doc['remixer_url']
                          }
                        )
    return played_docs
       

### Spot check parser

In [157]:
from urllib.request import Request, urlopen

url = 'https://www.1001tracklists.com/tracklist/2rrcqmpk/modd-vulcan-gas-company-austin-united-states-2019-03-09.html'
req = Request(url,\
              headers={'User-Agent': 'Mozilla/5.0'})
html = str(urlopen(req).read())

url_doc = {}
url_doc['html'] = html
url_doc.update(tracklist_meta_data(html))
url_doc.update(tracklist_general_information(html))
track_docs = tracklist_track_data(html)
url_doc['track_docs'] = track_docs

track_edges = build_track_edges(track_docs, url)
print('Length of track connections:', len(track_edges))
sequential_edges = build_sequential_track_edges(track_docs, url)
print('Length of Sequential connections:', len(sequential_edges))
played_edges = build_played_playedby_edge(url_doc, url)
print('Length of played connections:', len(played_edges))
artist_edges = build_artist_edges(url_doc, url)
print('Length of artist connections:', len(artist_edges))

Length of track connections: 91
Length of Sequential connections: 13
Length of played connections: 14
Length of artist connections: 85


### Run over these and keep a depth of like 2 for POC

In [ ]:
# import time

# def find_str(s, char, start_index=0):

#     index = 0
#     s = s[start_index+1:]
#     if char in s:
#         c = char[0]
#         for ch in s:
#             if ch == c:
#                 if s[index:index+len(char)] == char:
#                     return start_index + 1 + index
#             index += 1
#     return -1   

# def request(url):

#     user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
#     req = Request(url,\
#                   headers={'User-Agent': user_agent})
#     html = str(urlopen(req).read())
#     return html

# date_pages = ["https://www.1001tracklists.com/index%d.html?order=date" %d for d in range(100)]
# urls = []
# for url in date_pages:
    
#     # Make http request
#     html = request(url)
#     index = 0
#     # Iterate over links found in html
#     count = 0
#     while find_str(html, 'a href="', index) != -1:
        
#         # Extract url
#         index = find_str(html, 'a href="', index)
#         url_chunk = html[index:].split('"')[1]

#         # Make sure it is either a referenced tracklist or 1001 page
#         if ('/tracklist/' in url_chunk) and ('http' not in url_chunk):
#             new_page = 'https://www.1001tracklists.com' + url_chunk
#             urls.append(new_page)
#         if ('www.1001tracklists.com' in url_chunk) and ('.html' in url_chunk):
#             urls.append(url_chunk)
            
#         count += 1
            
#     print('Number of URLS:', len(urls))
#     time.sleep(10)

# import pickle
# with open('1001_urls.pkl', 'wb') as f:
#      pickle.dump(urls, f)

In [11]:
# import numpy as np 
# import pickle

# with open('1001_urls.pkl', 'rb') as f:
#      urls = pickle.load(f)
        
# print(len(urls))
# unique_urls = np.unique(urls)
# print(len(unique_urls))
# unique_urls = [url for url in unique_urls if ('.html' in url) and ('#tlp' not in url)]
# print(len(unique_urls))

4534
2980
2970


In [ ]:
import time
import pickle
import numpy as np

def parse(url):
    
    req = Request(url,\
                  headers={'User-Agent': 'Mozilla/5.0'})
    html = str(urlopen(req).read())   
    
    url_doc = {}
    url_doc['url'] = url
    url_doc['html'] = html
    url_doc.update(tracklist_meta_data(html))
    url_doc.update(tracklist_general_information(html))

    track_docs = tracklist_track_data(html)
    url_doc['track_docs'] = track_docs

    track_edges = build_track_edges(track_docs, url)
    print('Length of track connections:', len(track_edges))
    sequential_edges = build_sequential_track_edges(track_docs, url)
    print('Length of Sequential connections:', len(sequential_edges))
    played_edges = build_played_playedby_edge(url_doc, url)
    print('Length of played connections:', len(played_edges))
    artist_edges = build_artist_edges(url_doc, url)
    print('Length of artist connections:', len(artist_edges))

    return url_doc, track_edges, sequential_edges, played_edges, artist_edges, html

    
played_docs = []
sequential_docs = []
track_docs = []
tracklist_docs = []
artist_docs = []
url_html_map = {}

with open('1001_urls.pkl', 'rb') as f:
    urls = pickle.load(f)

# Find unique urls
seen_urls = []
unique_urls = np.unique(urls)
unique_urls = [url for url in unique_urls\
                   if ('.html' in url) and ('#tlp' not in url) and (url not in seen_urls)]

for url in unique_urls[:]:
    
    if ('.html' in url) and ('#tlp' not in url) and (url not in seen_urls):
    
        try:
            
            url_doc, track_edges, sequential_edges, played_edges, artist_edges, html = parse(url)

            url_html_map[url] = html
            
            played_docs.extend(played_edges)
            sequential_docs.extend(sequential_edges)
            track_docs.extend(track_edges)
            tracklist_docs.append(url_doc)
            artist_docs.extend(artist_edges)

            print('Len played docs:', len(played_docs))
            print('Len sequential docs:', len(sequential_docs))
            print('Len track docs:', len(track_docs))
            print('Len tracklist docs:', len(tracklist_docs))
            print('Len artist docs:', len(artist_docs))
            
            with open('played_docs3.pkl', 'wb') as f:
                pickle.dump(played_docs, f)
            with open('sequential_docs3.pkl', 'wb') as f:
                pickle.dump(sequential_docs, f)
            with open('track_docs3.pkl', 'wb') as f:
                pickle.dump(track_docs, f)
            with open('tracklist_docs3.pkl', 'wb') as f:
                pickle.dump(tracklist_docs, f)
            with open('artist_docs3.pkl', 'wb') as f:
                pickle.dump(artist_docs, f)
            with open('url_html_map.pkl', 'wb') as f:
                pickle.dump(artist_docs, f)
                
            seen_urls.append(url)
            
        except Exception as e:
            print(e)
        
        print('waiting')
        time.sleep(10)

In [ ]:
# Crawl anjunadeep

crawler = Crawler()
anjuna_urls = crawler.start_crawl('https://www.1001tracklists.com/source/v7m7k3/the-anjunadeep-edition/index.html')

import pickle
with open('anjuna_urls.pkl', 'wb') as f:
     pickle.dump(anjuna_urls, f)

In [ ]:
import time
import pickle
import numpy as np

def parse(url):
    
    req = Request(url,\
                  headers={'User-Agent': 'Mozilla/5.0'})
    html = str(urlopen(req).read())   
    
    url_doc = {}
    url_doc['url'] = url
    url_doc['html'] = html
    url_doc.update(tracklist_meta_data(html))
    url_doc.update(tracklist_general_information(html))

    track_docs = tracklist_track_data(html)
    url_doc['track_docs'] = track_docs

    track_edges = build_track_edges(track_docs, url)
    print('Length of track connections:', len(track_edges))
    sequential_edges = build_sequential_track_edges(track_docs, url)
    print('Length of Sequential connections:', len(sequential_edges))
    played_edges = build_played_playedby_edge(url_doc, url)
    print('Length of played connections:', len(played_edges))
    artist_edges = build_artist_edges(url_doc, url)
    print('Length of artist connections:', len(artist_edges))

    return url_doc, track_edges, sequential_edges, played_edges, artist_edges, html

    
played_docs = []
sequential_docs = []
track_docs = []
tracklist_docs = []
artist_docs = []
url_html_map = {}

with open('1001_urls.pkl', 'rb') as f:
    urls = pickle.load(f)

# Find unique urls
seen_urls = []
unique_urls = np.unique(urls)
unique_urls = [url for url in unique_urls\
                   if ('.html' in url) and ('#tlp' not in url) and (url not in seen_urls)]

for url in unique_urls[:]:
    
    if ('.html' in url) and ('#tlp' not in url) and (url not in seen_urls):
    
        try:
            
            url_doc, track_edges, sequential_edges, played_edges, artist_edges, html = parse(url)

            url_html_map[url] = html
            
            played_docs.extend(played_edges)
            sequential_docs.extend(sequential_edges)
            track_docs.extend(track_edges)
            tracklist_docs.append(url_doc)
            artist_docs.extend(artist_edges)

            print('Len played docs:', len(played_docs))
            print('Len sequential docs:', len(sequential_docs))
            print('Len track docs:', len(track_docs))
            print('Len tracklist docs:', len(tracklist_docs))
            print('Len artist docs:', len(artist_docs))
            
            with open('played_docs3.pkl', 'wb') as f:
                pickle.dump(played_docs, f)
            with open('sequential_docs3.pkl', 'wb') as f:
                pickle.dump(sequential_docs, f)
            with open('track_docs3.pkl', 'wb') as f:
                pickle.dump(track_docs, f)
            with open('tracklist_docs3.pkl', 'wb') as f:
                pickle.dump(tracklist_docs, f)
            with open('artist_docs3.pkl', 'wb') as f:
                pickle.dump(artist_docs, f)
            with open('url_html_map.pkl', 'wb') as f:
                pickle.dump(artist_docs, f)
                
            seen_urls.append(url)
            
        except Exception as e:
            print(e)
        
        print('waiting')
        time.sleep(10)